## Importar librerías

In [39]:
import os
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.decomposition import PCA, KernelPCA, TruncatedSVD, FactorAnalysis, FastICA
import umap
from sklearn_genetic import GASearchCV
from sklearn_genetic.space import Integer, Categorical, Continuous
from sklearn_genetic.callbacks import ProgressBar
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.pipeline import Pipeline
from skl2onnx.common.data_types import FloatTensorType
from skl2onnx import convert_sklearn, update_registered_converter
from skl2onnx.common.shape_calculator import calculate_linear_classifier_output_shapes
from onnxmltools.convert.xgboost.operator_converters.XGBoost import convert_xgboost
from onnxmltools.convert import convert_xgboost
import warnings
warnings.filterwarnings('ignore')

In [40]:
# Cargar
tester_files_folder = r"C:\Users\Administrador\AppData\Roaming\MetaQuotes\Terminal\Common\Files"
df_buy = pd.read_csv(os.path.join(tester_files_folder, "buy_training_dataset.csv"))
df_sell = pd.read_csv(os.path.join(tester_files_folder, "sell_training_dataset.csv"))
print(f"Buy -> Trades: {df_buy.shape[0]} Features: {df_buy.shape[1]-1}")
print(f"Sell -> Trades: {df_sell.shape[0]} Features: {df_sell.shape[1]-1}")

Buy -> Trades: 3810 Features: 296
Sell -> Trades: 3456 Features: 296


In [41]:
df_buy

,data_0,data_1,data_2,data_3,data_4,data_5,data_6,data_7,data_8,data_9,...,data_287,data_288,data_289,data_290,data_291,data_292,data_293,data_294,data_295,target
0,0.608491,1.090383,1.150728,-0.000116,0.000240,36.0,-0.000194,0.000243,43.0,-0.000194,...,47.0,-0.000194,0.000247,57.0,-0.000116,0.000239,74.0,0.916667,0.833333,1.0
1,-1.093850,0.571160,3.638356,-0.000542,0.000382,36.0,-0.000446,0.000355,31.0,-0.000574,...,15.0,-0.000191,0.000420,21.0,-0.000574,0.000406,39.0,0.208333,0.500000,1.0
2,-1.285653,1.096313,8.977362,-0.001398,0.001097,458.0,-0.001591,0.000997,393.0,-0.002920,...,605.0,0.000081,0.000572,657.0,0.000711,0.000522,211.0,0.458333,0.833333,1.0
3,-0.120234,-0.835369,3.591810,-0.000319,0.000365,121.0,-0.000255,0.000365,95.0,-0.000511,...,144.0,0.000000,0.000551,121.0,-0.000128,0.000574,74.0,0.791667,0.166667,0.0
4,1.205040,3.277762,7.948585,-0.000869,0.002090,143.0,-0.001313,0.002049,103.0,-0.003332,...,161.0,-0.004454,0.001687,225.0,0.003058,0.001541,299.0,0.333333,0.666667,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3805,-0.390148,-0.459753,5.649565,-0.000638,0.000994,263.0,-0.000766,0.001035,239.0,-0.001276,...,245.0,-0.001212,0.001272,303.0,-0.000766,0.001278,318.0,0.500000,0.166667,1.0
3806,-0.179353,0.957027,1.985179,-0.000054,0.000259,17.0,-0.000108,0.000255,22.0,-0.000163,...,15.0,-0.000488,0.000327,10.0,-0.000488,0.000344,27.0,0.250000,0.500000,1.0
3807,-1.008063,0.142789,10.194604,-0.001270,0.000566,45.0,-0.001758,0.000673,21.0,-0.001563,...,38.0,-0.002344,0.001178,31.0,-0.002344,0.001201,49.0,0.166667,0.333333,1.0
3808,-0.158214,-1.034292,10.091886,-0.001046,0.000982,317.0,-0.001793,0.000917,297.0,-0.001495,...,260.0,-0.001569,0.000570,262.0,-0.001465,0.000550,246.0,0.708333,0.500000,0.0


In [42]:
df_sell

,data_0,data_1,data_2,data_3,data_4,data_5,data_6,data_7,data_8,data_9,...,data_287,data_288,data_289,data_290,data_291,data_292,data_293,data_294,data_295,target
0,-0.085393,-0.502097,2.995601,0.000190,0.000673,188.0,0.000418,0.000698,272.0,0.000456,...,334.0,0.001027,0.001272,325.0,0.000951,0.001324,286.0,0.833333,0.333333,1.0
1,-0.113953,-0.758660,15.020192,0.001140,0.001257,290.0,0.002154,0.001317,256.0,0.002599,...,469.0,0.003932,0.001713,418.0,0.003742,0.001682,557.0,0.458333,0.666667,0.0
2,0.374872,-0.035757,2.813820,0.000261,0.000604,165.0,0.000457,0.000599,171.0,0.000523,...,174.0,0.000425,0.000595,130.0,0.000359,0.000590,162.0,0.916667,0.500000,1.0
3,-0.564332,1.744813,10.778267,0.001403,0.000827,507.0,0.001372,0.000736,719.0,0.001357,...,177.0,0.002033,0.000568,144.0,0.001539,0.000538,204.0,0.416667,0.166667,0.0
4,0.788140,0.511781,8.146138,0.000851,0.001648,316.0,0.002120,0.001618,358.0,0.001939,...,251.0,0.007085,0.001798,222.0,0.008079,0.001852,299.0,0.583333,0.333333,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3451,0.956417,1.330995,3.130846,0.000345,0.000793,245.0,0.000552,0.000817,218.0,0.000896,...,133.0,0.000689,0.000852,235.0,-0.000069,0.000877,248.0,0.708333,0.833333,1.0
3452,1.210330,1.052466,7.174148,0.000932,0.000511,57.0,0.001336,0.000456,43.0,0.000994,...,10.0,0.000528,0.000317,16.0,0.000528,0.000326,2.0,0.333333,0.500000,0.0
3453,1.105456,0.578609,55.290406,0.010972,0.002629,734.0,0.009452,0.001967,143.0,0.008118,...,48.0,0.006687,0.001495,58.0,0.008778,0.001526,146.0,0.375000,0.166667,0.0
3454,1.121175,0.693883,4.675750,0.000773,0.000398,22.0,0.001074,0.000340,22.0,0.000988,...,10.0,0.001890,0.000500,14.0,0.001847,0.000525,13.0,0.166667,0.500000,0.0


In [43]:
# Chequear y limpiar
if(df_buy.isna().values.any()):
    df_buy=df_buy.dropna()
if(df_sell.isna().values.any()):
    df_sell=df_sell.dropna()

In [44]:
# Dividir 
# Preparación de los datos de compra
X_buy = df_buy.drop(columns='target').values
y_buy = df_buy['target'].values
X_buy_train = X_buy
y_buy_train = y_buy
# Preparación de los datos de venta
X_sell = df_sell.drop(columns='target').values
y_sell = df_sell['target'].values
X_sell_train = X_sell
y_sell_train = y_sell

In [45]:
class DimensionalityReducer(BaseEstimator, TransformerMixin):
    def __init__(self, method=None, n_components=None, kernel=None,
             n_neighbors=None, min_dist=None):
        self.method = method
        self.n_components = n_components
        self.kernel = kernel
        self.n_neighbors = n_neighbors
        self.min_dist = min_dist
        self.reducer = None

    def fit(self, X, y=None):
        n_components = self._get_n_components(X)
        if self.method == 'pca':
            self.reducer = PCA(n_components=n_components)
        elif self.method == 'kernel_pca':
            kernel = self.kernel if self.kernel else 'linear'
            self.reducer = KernelPCA(n_components=n_components, kernel=kernel)
        elif self.method == 'truncated_svd':
            self.reducer = TruncatedSVD(n_components=n_components)
        elif self.method == 'factor_analysis':
            self.reducer = FactorAnalysis(n_components=n_components)
        elif self.method == 'fast_ica':
            self.reducer = FastICA(n_components=n_components, random_state=42)
        elif self.method == 'umap':
            n_neighbors = self.n_neighbors if self.n_neighbors else 15
            min_dist = self.min_dist if self.min_dist else 0.1
            self.reducer = umap.UMAP(n_components=n_components, n_neighbors=n_neighbors,
                                    min_dist=min_dist, random_state=42)

        if self.reducer is not None:
            self.reducer.fit(X, y)
        return self

    def transform(self, X):
        if self.reducer is not None:
            return self.reducer.transform(X)
        else:
            return X

    def _get_n_components(self, X):
        if self.n_components is None:
            return None
        if self.method == 'pca':
            # Para PCA, pasamos n_components directamente
            return self.n_components
        else:
            # Para otros métodos, aseguramos que sea un entero válido
            n_features = X.shape[1]
            if isinstance(self.n_components, float):
                n_components = int(self.n_components * n_features)
            else:
                n_components = self.n_components
            n_components = max(1, min(n_components, n_features))
            return n_components

In [46]:
# Número de características
n_features = X_buy_train.shape[1]
param_grid = {
    # Escalador
    'scaler': Categorical([StandardScaler(), RobustScaler()]),
    # Método de reducción de dimensionalidad
    'dim_reducer__method': Categorical(['pca', 'kernel_pca', 'truncated_svd', 'factor_analysis',
                                        'fast_ica', 'umap']),
    # Hiperparámetros comunes
    'dim_reducer__n_components': Continuous(0.5, 0.95),
    # Hiperparámetros específicos
    'dim_reducer__kernel': Categorical(['linear', 'poly', 'rbf', 'sigmoid', 'cosine']),
    'dim_reducer__n_neighbors': Integer(5, 50),
    'dim_reducer__min_dist': Continuous(0.0, 0.5),
    # Hiperparámetros de XGBoost
    'xgb__n_estimators': Integer(50, 500),
    'xgb__max_depth': Integer(3, 10),
    'xgb__learning_rate': Continuous(0.01, 0.3),
    'xgb__subsample': Continuous(0.6, 1.0),
    'xgb__colsample_bytree': Continuous(0.6, 1.0),
    'xgb__gamma': Continuous(0, 0.5),
    'xgb__min_child_weight': Integer(1, 10),
    'xgb__reg_alpha': Continuous(0, 1.0),
    'xgb__reg_lambda': Continuous(0, 1.0)
}

In [ ]:
# Definir el pipeline con placeholders (compras)
pipe_buy = Pipeline([
    ('scaler', RobustScaler()),
    ('dim_reducer', DimensionalityReducer()),
    ('xgb', xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42))
])
evolutionary_search_buy = GASearchCV(
    estimator=pipe_buy,
    cv=5,
    scoring='accuracy',
    param_grid=param_grid,
    population_size=50,
    generations=20,
    mutation_probability=0.1,
    crossover_probability=0.9,
    n_jobs=-1,
    verbose=True,
    keep_top_k=10
)
# Entrenar el modelo
evolutionary_search_buy.fit(X_buy_train, y_buy_train, callbacks=[ProgressBar()])
# Obtener el mejor estimador
model_buy = evolutionary_search_buy.best_estimator_

  0%|          | 0/21 [00:00<?, ?it/s]

In [ ]:
# Crear una figura con dos subgráficos (2 filas, 1 columna)
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))  # Ajusta el tamaño según prefieras
# Primer subgráfico: Evolución del Fitness
plt.sca(ax1)
evolutionary_search_buy.plot_fitness_evolution()
ax1.set_title('Evolución del Fitness - Modelo de Compras')
# Segundo subgráfico: Importancia de los Hiperparámetros
plt.sca(ax2)
evolutionary_search_buy.plot_parameters_importance()
ax2.set_title('Importancia de los Hiperparámetros - Modelo de Compras')
# Ajustar el espaciado entre los subgráficos
plt.tight_layout()
# Mostrar los gráficos
plt.show()
# Score
print(f'Model sell score: {evolutionary_search_buy.best_score_}')

In [ ]:
# Definir el pipeline con placeholders (compras)
pipe_sell = Pipeline([
    ('scaler', 'passthrough'),
    ('dim_reducer', DimensionalityReducer()),
    ('xgb', xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42))
])
# Configurar la búsqueda genética
evolutionary_search_sell = GASearchCV(
    estimator=pipe_sell,
    cv=5,
    scoring='accuracy',
    param_grid=param_grid,
    population_size=50,
    generations=20,
    mutation_probability=0.1,
    crossover_probability=0.9,
    n_jobs=-1,
    verbose=True,
    keep_top_k=10
)
# Entrenar el modelo
evolutionary_search_sell.fit(X_sell_train, y_sell_train, callbacks=[ProgressBar()])
# Obtener el mejor estimador
model_sell = evolutionary_search_sell.best_estimator_

In [ ]:
# Crear una figura con dos subgráficos (2 filas, 1 columna)
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))  # Ajusta el tamaño según prefieras
# Primer subgráfico: Evolución del Fitness
plt.sca(ax1)
evolutionary_search_sell.plot_fitness_evolution()
ax1.set_title('Evolución del Fitness - Modelo de Ventas')
# Segundo subgráfico: Importancia de los Hiperparámetros
plt.sca(ax2)
evolutionary_search_sell.plot_parameters_importance()
ax2.set_title('Importancia de los Hiperparámetros - Modelo de Ventas')
# Ajustar el espaciado entre los subgráficos
plt.tight_layout()
# Mostrar los gráficos
plt.show()
# Score
print(f'Model sell score: {evolutionary_search_sell.best_score_}')

In [10]:
update_registered_converter(
    xgb.XGBClassifier,
    "XGBClassifier",
    calculate_linear_classifier_output_shapes,
    convert_xgboost,
    options={'nocl': [False], 'zipmap': [True, False, 'columns']}
)

In [11]:
model_buy_onnx = convert_sklearn(
    model_buy,
    'pipeline_buy_xgboost',
    [('input', FloatTensorType([None, X_buy_train.shape[1]]))],
    target_opset={'': 12, 'ai.onnx.ml': 2}
)
model_sell_onnx = convert_sklearn(
    model_sell,
    'pipeline_sell_xgboost',
    [('input', FloatTensorType([None, X_buy_train.shape[1]]))],
    target_opset={'': 12, 'ai.onnx.ml': 2}
)
mql5_files_folder = "C:/Users/Administrador/AppData/Roaming/MetaQuotes/Terminal/6C3C6A11D1C3791DD4DBF45421BF8028/MQL5/Files"
with open(os.path.join(mql5_files_folder, "model_buy.onnx"), 'wb') as f:
    f.write(model_buy_onnx.SerializeToString())
with open(os.path.join(mql5_files_folder, "model_sell.onnx"), 'wb') as f:
    f.write(model_sell_onnx.SerializeToString())